<a href="https://colab.research.google.com/github/chaerui7967/Netflix_Analysis_individual/blob/master/%EC%A0%90%EC%88%98_%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%ACipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


import matplotlib.pyplot as plt
import missingno # 널값 바차트로 시각화

In [ ]:
netflix = pd.read_csv('./use_df.csv')

In [ ]:
netflix.head(1)

In [ ]:
netflix.columns

In [ ]:
df_net = netflix.drop(columns = ['Unnamed: 0'],axis =1)

In [ ]:
df_net.isnull().sum()

In [ ]:
color= ['dimgrey','dimgrey','blue','blue','blue','dimgrey','blue','blue','blue','blue','blue',
        'dimgrey','blue','blue','blue','blue','blue','blue','blue','blue','blue','blue','dimgrey',
        'blue','blue','blue']
missingno.bar(df_net,fontsize=10,color=color,figsize=(10,5))
plt.title('COLUMN MISSING VALUES',fontsize=20)

viewpoint 예측에 사용될 변수들을 일단은 국가와 장르, 태그, 언어, 출시 일, 런타임을 가지고 먼저 예측초기모델을 만들어보자

null값 처리

In [ ]:
df = df_net[['Series or Movie','COUNTRY', 'Release Date', 'GENRE', 'Tags', 'Languages', 'Runtime']]

In [ ]:
df_taget = df_net['VALUE']

In [ ]:
df.isnull().sum()

In [ ]:
df['Release Date'] = df['Release Date'].fillna(0)

In [ ]:
df = df.fillna('NA')  # null값을 모두 'NA'으로 치환

In [ ]:
df.isnull().sum()

### Release Date는 년도만 사용

In [ ]:
df["Release Date"] = pd.to_datetime(df['Release Date'])
df['Year'] = df['Release Date'].dt.year

In [ ]:
df = df.drop(columns = ['Release Date'],axis =1)

In [ ]:
df.head(1)

# 변수들을 원핫 인코딩으로 인코딩

In [ ]:
pd_df = pd.get_dummies(df)

## 딥러닝을 통해 모든 변수들을 넣고 예측

In [ ]:
from keras import models
from keras import layers

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(pd_df.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
import numpy as np

k = 4

num_val_samples = len(pd_df) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = pd_df[i * num_val_samples: (i + 1) * num_val_samples]  # 검증 데이터 준비: k번째 분할
    val_targets = df_taget[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [pd_df[:i * num_val_samples],
         pd_df[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [df_taget[:i * num_val_samples],
         df_taget[(i + 1) * num_val_samples:]],
        axis=0)

model = build_model()  # 케라스 모델 구성(컴파일 포함)
model.fit(partial_train_data, partial_train_targets,  # 모델 훈련(verbose=0이므로 훈련 과정이 출력되지 않습니다.)
          epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)  # 검증 세트로 모델 평가
all_scores.append(val_mae)

In [ ]:
all_scores

**에포크 100으로 딥러닝을 진행했을 때 오차가 490으로 viewpoint가 1~40000인것을 감안하면 오차가 좀 큰걸로 나옴**
- all_scores = [490.316650390625]

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = pd_df[i * num_val_samples: (i + 1) * num_val_samples]  #검증 데이터 준비: k번째 분할    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [pd_df[:i * num_val_samples],
         pd_df[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [df_taget[:i * num_val_samples],
         df_taget[(i + 1) * num_val_samples:]],
        axis=0)

model = build_model()  # 케라스 모델 구성(컴파일 포함)
history = model.fit(partial_train_data, partial_train_targets,  # 모델 훈련(verbose=0이므로 훈련 과정이 출력되지 않습니다.)
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=1, verbose=0)
# mae_history = history.history['val_mae']
# all_mae_histories.append(mae_history)

In [ ]:
mae_history = history.history['val_mae']
all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = build_model()  # 새롭게 컴파일된 모델을 얻습니다.
model.fit(pd_df, df_taget,  # 전체 데이터로 훈련시킵니다.
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(pd_df, df_taget)

94/94 [==============================] - 1s 2ms/step - loss: 4435821.0000 - mae: 753.8995

test_mae_score = 753.8995361328125

In [ ]:
test_mae_score

In [ ]:
# pd_df = pd.get_dummies(df_net['Tags'])
# df_result = pd.concat([df_net, pd_df], axis=1)